In [118]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import time
import re
import urllib
import os

In [78]:
url_home = 'http://www.dmm.co.jp'
url_base = 'http://www.dmm.co.jp/digital/videoa/-/ranking/=/term=monthly/type=actress/page='
pages = range(1,6,1)


In [129]:
acts = []
for p in pages:
    url = url_base + str(p) + '/'
    soup = BeautifulSoup(requests.get(url).content) 
    rank = len(soup.find_all('table',{'class':"w100 mg-b20 work"})) 
    for r in range(rank):  
        for a in range(2):
            img_tag = soup.find_all('table',{'class':"w100 mg-b20 work"})[r].find_all('td')[a]
            
            face_url =img_tag.find_all('a')[0].find('img')['src']
            face_name = img_tag.find_all('a')[0].find('img')['src'].split('/')[-1]
            name = img_tag.find_all('a')[0].find('img')['alt']
            
            url_act = url_home + img_tag.find_all('a')[1]['href']
            soup_act = BeautifulSoup(requests.get(url_act).content) 
            num_titles = len(soup_act.find('div', {'class':"d-item"}).find_all('p', {'class':'tmb'}))
            
            for i in range(num_titles):
                url_sample = soup_act.find('div', {'class':"d-item"}).find_all('p', {'class':'tmb'})[i].find('a')['href'].split('?')[0]
                soup_sample = BeautifulSoup(requests.get(url_sample).content) 
                num_appear = len(soup_sample.find('span', {'id':'performer'}))
                title = soup_act.find('div', {'class':"d-item"}).find_all('p', {'class':'tmb'})[i].find('img')['alt']
                if num_appear > 2:
                    continue
                if soup_sample.find('div', {'id':"sample-image-block"}) is None:
                    continue
                    
                num_samples = len(soup_sample.find('div', {'id':"sample-image-block"}).find_all('img'))
                for num in range(num_samples):
                    img_url = soup_sample.find('div', {'id':"sample-image-block"}).find_all('img')[0]['src'].split('-')
                    img_url_rev = img_url[0] + 'jp-' + str(num+1) + '.jpg'
                    file_name = img_url_rev.split('/')[-1]

                    act = {'name':name,
                           'face_url':face_url,
                           'face_name':face_name,
                           'img_url':img_url_rev,
                           'file_name':file_name,
                           'title':title}
                    acts.append(act)
                    
                    act_dir = 'images_best_titles/' + name
                    
                    try:
                        os.makedirs(act_dir)
                    except FileExistsError:
                        pass
                    
                    dst_path = act_dir + '/' + file_name
                    data = urllib.request.urlopen(img_url_rev).read()
                    with open(dst_path, mode="wb") as f:
                        f.write(data)
                    time.sleep(0.5)

In [130]:
df = pd.DataFrame(acts)
df.to_csv('acts_best_titles.csv', index=False)

In [131]:
pd.DataFrame(acts)

,face_name,face_url,file_name,img_url,name,title
0,mikami_yua.jpg,http://pics.dmm.co.jp/mono/actjpgs/medium/mika...,ssni00409jp-1.jpg,http://pics.dmm.co.jp/digital/video/ssni00409/...,三上悠亜,秘密捜査官の女 媚薬漬け限界拷問スペシャル 三上悠亜
1,mikami_yua.jpg,http://pics.dmm.co.jp/mono/actjpgs/medium/mika...,ssni00409jp-2.jpg,http://pics.dmm.co.jp/digital/video/ssni00409/...,三上悠亜,秘密捜査官の女 媚薬漬け限界拷問スペシャル 三上悠亜
2,mikami_yua.jpg,http://pics.dmm.co.jp/mono/actjpgs/medium/mika...,ssni00409jp-3.jpg,http://pics.dmm.co.jp/digital/video/ssni00409/...,三上悠亜,秘密捜査官の女 媚薬漬け限界拷問スペシャル 三上悠亜
3,mikami_yua.jpg,http://pics.dmm.co.jp/mono/actjpgs/medium/mika...,ssni00409jp-4.jpg,http://pics.dmm.co.jp/digital/video/ssni00409/...,三上悠亜,秘密捜査官の女 媚薬漬け限界拷問スペシャル 三上悠亜
4,mikami_yua.jpg,http://pics.dmm.co.jp/mono/actjpgs/medium/mika...,ssni00409jp-5.jpg,http://pics.dmm.co.jp/digital/video/ssni00409/...,三上悠亜,秘密捜査官の女 媚薬漬け限界拷問スペシャル 三上悠亜
5,mikami_yua.jpg,http://pics.dmm.co.jp/mono/actjpgs/medium/mika...,ssni00409jp-6.jpg,http://pics.dmm.co.jp/digital/video/ssni00409/...,三上悠亜,秘密捜査官の女 媚薬漬け限界拷問スペシャル 三上悠亜
6,mikami_yua.jpg,http://pics.dmm.co.jp/mono/actjpgs/medium/mika...,ssni00409jp-7.jpg,http://pics.dmm.co.jp/digital/video/ssni00409/...,三上悠亜,秘密捜査官の女 媚薬漬け限界拷問スペシャル 三上悠亜
7,mikami_yua.jpg,http://pics.dmm.co.jp/mono/actjpgs/medium/mika...,ssni00409jp-8.jpg,http://pics.dmm.co.jp/digital/video/ssni00409/...,三上悠亜,秘密捜査官の女 媚薬漬け限界拷問スペシャル 三上悠亜
8,mikami_yua.jpg,http://pics.dmm.co.jp/mono/actjpgs/medium/mika...,ssni00409jp-9.jpg,http://pics.dmm.co.jp/digital/video/ssni00409/...,三上悠亜,秘密捜査官の女 媚薬漬け限界拷問スペシャル 三上悠亜
9,mikami_yua.jpg,http://pics.dmm.co.jp/mono/actjpgs/medium/mika...,ssni00409jp-10.jpg,http://pics.dmm.co.jp/digital/video/ssni00409/...,三上悠亜,秘密捜査官の女 媚薬漬け限界拷問スペシャル 三上悠亜
